# Manhattan Rental Apartments Clustering
## - Model Selection

### Xian Lai
xlai4@fordham.edu   
Apr.2017

=======================================================
<img src="images/title_image.jpg" width="1200">

### Abstract:
A city functions like a gigantic sophisticate network. Within it each buildings and blocks are connected by visible transportation systems and invisible functional dependencies. But on the other hand, the difference of locations and functionality also divides the city into many sub-areas. Forming by different causes, the boundaries of these sub-areas are different. Like for political administration, we have boroughs, community districts and neighbourhoods, and for postal service, we have zip codes. 

In this projet, I would like to make use of rental apartment online listing dataset and new york building footprint dataset to explore the possible geographic boundaries or patterns of rental apartments demands in Manhattan.

To extract the structure of demand pattern, we will use unsupervised clustering technique to find the best grouping of buildings with respect to their location, their facility properties and popularity.

In detail, we are going to:
- Interpolate the popularity of every building in the building dataset.
- Select the best clustering model using a sample dataset.
- Perform the final model on the full dataset and query the information we want.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import ParameterGrid
import sys; sys.path.append('modules')
import HierarchicalClustering as hc
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import Spectral11 as cm
output_notebook()
pd.options.mode.chained_assignment = None

Loading BokehJS ...

## 0. Data sets
As previously mentioned, in this project, I am using 2 datasets: 
- **listing dataset**
- **building dataset** 

The first one contains about 46,000 rental apartment online listings in Manhattan. Each listing containing information about rental apartments’ geographic location, popularity (defined by how many visits of listing webpage) and some other description features like facilities, number of bedrooms, bathrooms, rental price, etc.

But this listing dataset only covers part of buildings in Manhattan and each listing is about one apartment rather than building. So I bring in the 2nd dataset which simply consists of every building in Manhattan and their locations. 

Assuming the popularity of rental apartment is geographically continuous, namely the popularity of one building is similar to surrounding buildings, I can interpolate the popularities of every building in apartment rental market using the information from listing dataset. And then I will perform clustering on building dataset instead.

In [2]:
df_listing = pd\
    .read_pickle('data/market_set_cleaned.pickle')\
    .sample(n=10000)\
    .reset_index(drop=True)
df_listing.head()

,bath,bed,y,x,center_pt,price_bin,broker,elevator,fitness_center,popularity
0,1.0,2,356168.463866,-9.610533e+06,"(-9610533.388396502, 356168.46386598656)",1,0,0,0,2
1,1.0,3,362650.148748,-9.614508e+06,"(-9614508.125254028, 362650.1487481855)",3,1,1,1,1
2,1.0,0,359796.471599,-9.612532e+06,"(-9612531.849026758, 359796.47159874206)",1,0,1,1,1
3,2.5,4,359414.429051,-9.617285e+06,"(-9617284.522746988, 359414.4290507063)",3,0,1,1,1
4,1.0,3,359572.151837,-9.617575e+06,"(-9617575.466938905, 359572.15183659224)",3,0,1,1,1


In [3]:
df_building = pd\
    .read_pickle('data/building_set_cleaned.pickle')\
    .drop(labels=['DOITT_ID', 'footprint'], axis=1)\
    .sample(n=10000)\
    .reset_index(drop=True)
df_building.head()

,center_pt,x,y
0,"(-9617555.55107953, 366609.10212470964)",-9.617556e+06,366609.102125
1,"(-9613243.596262012, 355018.3241677196)",-9.613244e+06,355018.324168
2,"(-9614427.33816702, 359771.373410237)",-9.614427e+06,359771.373410
3,"(-9616075.98871911, 358152.0620266632)",-9.616076e+06,358152.062027
4,"(-9612895.453442965, 354136.5500597288)",-9.612895e+06,354136.550060


## 1. Interpolation and clustering

### Interpolation:
Based on assumption that popularity of buildings are similar to their surrounding buildings', I use inverse distance weighting (IDW) as my interpolation method to get popularity value for each data point in building dataset.

To interpolate the popularity for a building $y_k$ in building dataset, I first take **n closest apartments** in the listing dataset denoted as ${x_1, x_2, ..., x_I}$. And then calculate the inverse distance weighted average of popularities of ${x_1, x_2, ..., x_i}$ using **controlling power p** and assign the result as the popularity of $y_k$:

Assuming $u(x_i)$ is the popularity of $x$. Then we have the calculation formula:

If $d(y_k,x_i) \neq 0$ for all $x_i$:
$$u(y_k)=\frac{\sum_{i=1}^{I} w(x_i)*u(x_i)}{\sum_{i=1}^{I} w(x_i)}$$
where 
$$w(x_i)=\frac{1}{d(y_k,x_i)^p}$$
If $d(y_k,x_i) = 0$ for some $x_i$:
$$u(y_k)=u(x_i)$$

The intuitive understanding of p is the controlling power of closest points. Larger value of p allows closer apartments weight more in the interpolation. 

### Clustering:
With every building assigned popularity values, I performed hierarchical clustering using their longitude, latitude and the popularity. The reason to use hierarchical clustering is that we can choose different cuts on dendrogram, namely the number of clusters, so we gain different points of view from larger areas to small neighbourhoods.

For the clustering model, there are 2 hyperparameters to decide:
- **method**: Method to calculate distance between clusters.
- **metric**: Metric to calculate distance between data points.

Until now, we have 4 hyperparameters for our model: 
- n_beighbors : {5, 8, 13}
- IDWpower : {0.1, 0.5, 1.0, 3.0}
- method : {'average', 'weighted', 'complete', 'centroid'}
- metric : {'cityblock', 'euclidean'}

Because centroid method can only work with Euclidean metric, there are 84 combination of hyperparameters in total.

\*Clustering on large number of data points are computational expensive, for model selection, I use 10,000 data points sampled out of original building data set.

### Evalution:
To select the best one from these 84 models, we use the following 6 criteria:
1. n_singlton : The number of singleton clusters.
2. smClusterSize: The cluster size at the 15th percentile ranking from small to big.
3. lgClusterSize: The cluster size at the 85th percentile ranking from small to big.
4. lgClusterArea: The cluster area at the 85th percentile ranking from small to big.
5. interVariance: The within cluster popularity variance.
6. intraVariance: The between cluster popularity variance.

The first 4 criteria evaluate whether a model yields balanced clustering with respect to both size and area. For this application, we don't want the clustering with a few large clusters and many small clusters.

The last 2 criteria evaluate whether a model put nearby buildings with similar popularity in the same cluster and the ones with different popularity into different clusters.

In [4]:
"""
# In total there are 2 type of hyperparameters and 12 combinations
# for interpolation. So we use a list HCs to save all 12 possible
# hierarchical clusterings.
HCs  = []
params = {
    'n_ngb':[5, 8, 13],
    'IDWpower':[0.1, 0.5, 1.0, 3.0],
}
paramsGrid = list(ParameterGrid(params))
cnt = 0
for param in paramsGrid:
    cnt += 1; print(cnt)
    HC = hc.HierarchicalClustering(df_listing, df_building)
    HC.prepareData(**param)
    HCs.append((param, HC))

# There are again 2 types of hyperparameters and in total 7 
# combinations for clustering. We will apply each of them onto
# each of the hierarchical clustering instances produced above.
stats  = []
params = {
    'method':['average', 'weighted', 'complete'],
    'metric':['cityblock', 'euclidean'],
}
paramsGrid = list(ParameterGrid(params))
paramsGrid = paramsGrid.append({
    'method':'centroid',
    'metric':'euclidean'
})
for param in paramsGrid:
    for param_, HC in HCs:
        param_.update(param)
        HC.clustering(n_clusters=600, **param)
        stat = HC.clusteringStats()
        stat.update({'param':param_})
        stats.append(stat)
    
df_perf = pd.DataFrame(stats)
df_perf.to_csv("data/model_performances.csv", index=False)
"""
df_perf = pd.read_csv("data/model_performances.csv")
df_perf = df_perf.set_index('param')
df_perf.head()

,interVariance,intraVariance,lgClusterArea,lgClusterSize,n_singlton,smClusterSize
param,,,,,,
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'cityblock', 'n_ngb': 5}",-0.009260,2.572693,-1.027000,-203.60,-4,5.85
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'cityblock', 'n_ngb': 8}",-0.029241,2.677898,-1.106668,-191.75,-2,4.00
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'cityblock', 'n_ngb': 13}",-0.040542,3.324252,-0.895748,-209.30,-4,4.85
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'euclidean', 'n_ngb': 5}",-0.011875,2.627210,-1.080736,-192.60,-4,6.85
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'euclidean', 'n_ngb': 8}",-0.035000,2.841335,-0.929767,-191.50,-4,4.00


## 2. Clustering models evaluation and selection
Gaining the evaluation decisions given by 6 criteria for each clustering model, we need to find a way to combine them. There exist several different ways of combining the outputs of the scoring systems, including score combination, rank combination, voting, average combination, weighted combination etc. 

Here I use a scoring systems combination method introduced by Hsu and Taksa*:  
Let $S_m(n)$ and $R_m(n)$ be the score and rank given by $m^{th}$ criterion on $n^{th}$ model respectively. We will have $S_m(n) \in [0,1]$ with highest scoring = 1 and $R_m(n) \in [1,N]$ with highest ranking = 1. Then we can investigate the scoring behavior of different criterions defined by Rank-Score Characteristic(RSC):
$$RSC_m(n) = \frac{S_m(n)}{R_m(n)}$$

The RSC curves of each criterion will form rank-score graph that tells us how different each criterion deciding their scoring. The following picture is an illustration of 3 scoring systems. The scoring system who assigns scores in a linearly decreasing fashion will have a linear rank-score curve like $f_2$ does. The system who habitually assigns high scores to a large subset of its top ranked candidates will have a graph that is not a straight line, but has a low slope for the top ranked candidates and a higher slope for the remainder similar to $f_3$. A third class of scoring behavior is exemplified by $f_1$. In this case, the expert habitually gives higher scores to a small subset of its top ranked candidates and much lower scores to the rest. 
<img src="images/RSC_graph.png" width="200">

Hsu and Taksa indicate that a diversity measure based on the rank-score graph can be used to determine whether a score or rank fusion will produce a better result. **When the rank-score graphs of two systems are very SIMILAR, then a Score Combination will produce the best fusion. When the rank-score graphs are very DIFFERENT, then a Rank Combination produces the better result.**

\* Hsu, D.F. and Taksa, I., Comparing rank and score combination methods for data fusion in information retrieval.

In [5]:
def rankFunction(sr):
    """ transform the values in the series to its
    ranks in ascending order.
    """
    return sr.rank(method='min', ascending=False)

def scoreFunction(sr):
    """ transform the values in the series to its
    score in range [0,1].
    """
    sr = sr - sr.min()
    return sr/sr.max()

def scoreRankFunction(sr):
    """ transform the values in the series to its
    Rank-Score Characteristic which is the ratio of
    score and rank.
    """
    return scoreFunction(sr)/rankFunction(sr)

df_rank  = df_perf.apply(rankFunction, axis=0)
df_score = df_perf.apply(scoreFunction, axis=0)
df_src   = df_perf.apply(scoreRankFunction, axis=0)
df_rank.head()

,interVariance,intraVariance,lgClusterArea,lgClusterSize,n_singlton,smClusterSize
param,,,,,,
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'cityblock', 'n_ngb': 5}",1.0,50.0,76.0,62.0,46.0,57.0
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'cityblock', 'n_ngb': 8}",10.0,41.0,83.0,40.0,29.0,71.0
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'cityblock', 'n_ngb': 13}",22.0,9.0,49.0,67.0,46.0,70.0
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'euclidean', 'n_ngb': 5}",3.0,45.0,82.0,45.0,46.0,50.0
"{'IDWpower': 0.1, 'method': 'average', 'metric': 'euclidean', 'n_ngb': 8}",13.0,26.0,64.0,39.0,46.0,71.0


In [6]:
p = figure(plot_width=500, plot_height=400, title="Rank-Score Graph")
p.xgrid.minor_grid_line_color = 'black'
p.xgrid.minor_grid_line_alpha = 0.1
for cnt, col in enumerate(df_src.columns):
    criterion = df_src[col]\
        .sort_values(ascending=False)\
        .reset_index(drop=True)
    p.line(
        criterion.index, criterion, line_width=2,
        color=cm[2*cnt], alpha=0.8, legend=col
    )
p.xaxis.axis_label = "ranking"
p.yaxis.axis_label = "scoring"
show(p) 

As we observed that these SRC curves for 6 criteria all have concave shape which means that they share similar scoring behaviors(gives higher scores to a small subset of its top ranked candidates and much lower scores to the rest). So I should **combine the scores** of 6 criteria to find the best clustering model.

Here I use Mahalanobis weighting to combine the scores.
$$SC(n)=\sum_{m=1}^{M} w_m*S_m(n)$$

where $$w_m=\frac{\frac{1}{\sigma_m^2}}{\sum_{1}^{M} \frac{1}{\sigma_m^2}}$$


In [7]:
def combineScore(df):
    """
    """
    precisions = [1/df[col].var() for col in df.columns]
    weights    = [pcs / sum(precisions) for pcs in precisions]
    return np.dot(df.values, np.array(weights).T.reshape(-1, 1))

df_score['combinedScore'] = combineScore(df_score)
df_score = df_score.sort_values('combinedScore', ascending=False)
df_score.head()

,interVariance,intraVariance,lgClusterArea,lgClusterSize,n_singlton,smClusterSize,combinedScore
param,,,,,,,
"{'IDWpower': 0.1, 'method': 'complete', 'metric': 'euclidean', 'n_ngb': 13}",0.424524,0.787636,0.869245,0.730236,0.8,0.527307,0.690201
"{'IDWpower': 0.5, 'method': 'complete', 'metric': 'cityblock', 'n_ngb': 13}",0.309695,0.402596,0.614599,0.889736,1.0,0.973635,0.671835
"{'IDWpower': 0.1, 'method': 'complete', 'metric': 'euclidean', 'n_ngb': 8}",0.374070,0.468182,0.890013,0.612344,1.0,0.704331,0.668512
"{'IDWpower': 0.5, 'method': 'complete', 'metric': 'euclidean', 'n_ngb': 5}",0.481474,0.204911,0.792610,0.757282,1.0,0.785311,0.665854
"{'IDWpower': 0.1, 'method': 'complete', 'metric': 'euclidean', 'n_ngb': 5}",0.272584,0.305270,1.000000,0.816227,1.0,0.527307,0.658975


In [8]:
print("The final hyperparameters:\n{}".format(df_score.index[0]))

The final hyperparameters:
{'IDWpower': 0.1, 'method': 'complete', 'metric': 'euclidean', 'n_ngb': 13}


So this is the final clustering model we select. We will go on perform clustering on full dataset and query information from the clustering in next jupyter notebook.